# count_profanity

In [ ]:
import numpy as np
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score

from sites.bookcave import bookcave

In [ ]:
min_len, max_len = 250, 7500
categories_mode = 'soft'

In [ ]:
inputs, Y, categories, category_levels =\
    bookcave.get_data({'tokens'},
                      min_len=min_len,
                      max_len=max_len,
                      categories_mode=categories_mode,
                      only_categories={bookcave.CATEGORY_INDEX_PROFANITY})
text_paragraph_tokens = [paragraph_tokens for paragraph_tokens, _ in inputs['tokens']]
y = Y[0]
len(text_paragraph_tokens), len(y)

In [ ]:
text_all_tokens = []
for paragraph_tokens in text_paragraph_tokens:
    all_tokens = []
    for tokens in paragraph_tokens:
        all_tokens.extend(tokens)
    text_all_tokens.append(all_tokens)
len(text_all_tokens)

Vectorize.

In [ ]:
# Sensitive terms.
terms = [
    # Unigrams
    'fuck', 'fucking', 'shit', 'ass', 'fucked',
    'bitch', 'damn', 'asshole', 'bullshit', 'hell',
    'goddamn', 'pissed', 'bastard', 'pussy', 'dick',
    'piss', 'cock', 'damned', 'bastards', 'dammit',
    'crap', 'whore', 'jesus', 'balls', 'christ',
    # Bi-grams
    'my god', 'the lord'
]
# The chi-squared scores as calculated by `correlated_words.ipynb`.
scores = [
    # Unigrams
    53.898025279581404, 52.030154888391806, 47.704418886478464, 34.55475551500284, 33.1162229977325,
    30.47933938866534, 29.788814670866852, 29.694698250911742, 27.984428132905137, 23.359744651755545,
    23.15129752085929, 22.50285572996259, 21.851090886939872, 20.909696770700794, 17.488034484581796,
    17.30660075173425, 17.142074011125963, 15.672210844229596, 13.872836790230444, 12.746700176411284,
    9.656894314631861, 9.501671303087658, 3.7650110677763946, 3.09802624783671, 2.8661986785983244,
    # Bi-grams
    13.953372314896928, 3.4680166228627614
]
assert(len(terms) == len(scores))

In [ ]:
def identity(v):
    return v

In [ ]:
vectorizer = CountVectorizer(
    preprocessor=identity,
    tokenizer=identity,
    analyzer='word',
    token_pattern=None,
    ngram_range=(1, 2),
    vocabulary=terms)

In [ ]:
X_sparse = vectorizer.fit_transform(text_all_tokens)
X = X_sparse.toarray()
X.shape

In [ ]:
X

In [ ]:
def get_y_pred(count, mode='soft'):
    if mode == 'hard':
        thresholds = [0, 5, 40, 100, 200, 500]
        profanity_levels = [0, 2, 3, 4, 5, 5, 6]
    elif mode == 'medium':
        thresholds = [0, 5, 40, 100, 500]
        profanity_levels = [0, 2, 3, 4, 5, 6]
    elif mode == 'soft':
        thresholds = [0, 5, 100]
        profanity_levels = [0, 1, 2, 3]
    else:
        raise ValueError('Unknown value for `mode`: {}'.format(mode))
    
    for i, threshold in enumerate(thresholds):
        if count <= threshold:
            return profanity_levels[i]
    return profanity_levels[-1]

In [ ]:
def print_metrics(flags):
    counts = np.dot(X, np.array(flags))    
    y_pred = [get_y_pred(count, mode=categories_mode) for count in counts]

    confusion = confusion_matrix(y, y_pred)
    print(confusion)
    
    accuracy = accuracy_score(y, y_pred)
    print('{:>20}: {:.4f}'.format('Accuracy', accuracy))
    precision_macro = precision_score(y, y_pred, average='macro')
    print('{:>20}: {:.4f}'.format('Precision (macro)', precision_macro))
    recall_macro = recall_score(y, y_pred, average='macro')
    print('{:>20}: {:.4f}'.format('Recall (macro)', recall_macro))
    f1_macro = f1_score(y, y_pred, average='macro')
    print('{:>20}: {:.4f}'.format('F1 (macro)', f1_macro))
    precision_weighted = precision_score(y, y_pred, average='weighted')
    print('{:>20}: {:.4f}'.format('Precision (weighted)', precision_weighted))
    recall_weighted = recall_score(y, y_pred, average='weighted')
    print('{:>20}: {:.4f}'.format('Recall (weighted)', recall_weighted))
    f1_weighted = f1_score(y, y_pred, average='weighted')
    print('{:>20}: {:.4f}'.format('F1 (weighted)', f1_weighted))
    
    return y_pred

In [ ]:
flags = [
    # Unigrams
    1, 1, 1, 1, 1,
    1, 0, 1, 1, 0,
    1, 1, 1, 1, 1,
    1, 1, 1, 1, 1,
    0, 0, 0, 0, 0,
    # Bi-grams
    1, 0
]
y_pred = print_metrics(flags)

In [ ]:
falses = [x for i, x in enumerate(X) if y[i] == 1 and y_pred[i] == 2]
false_totals = np.sum(falses, axis=0)
false_totals